# Answer the F Question

## 1. 导入所需要的库

In [31]:
# 基本库
import os
import datetime
import time

# 数据读取
import pandas as pd
import numpy as np

# 可视化
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
from matplotlib import cm

# 操作Excel表格
# 只用读取即可。
import xlrd



import tensorflow as tf

import dgl

## 2 定义公共函数

输出信息

In [ ]:
def DisplayStartInfo():
    """
    :description: 显示开始时间。
    :param None {type: None}: 
    :return startTime {type: datetime.datetime}{count: 1}: 返回当前时间。
    """
    print("-------------------------Start-------------------------")
    startTime = datetime.datetime.now()
    print(startTime.strftime('%Y-%m-%d %H:%M:%S'))
    return startTime


def DisplayCompletedInfo(startTime=datetime.datetime.now(), isDisplayTimeConsumed=False):
    """
    :description: 显示结束时间信息。
    :param startTime {type: datetime.datetime} {default: datetime.datetime.now()}:: 显示开始时间。default值搭配着isDisplayTimeConsumed=False一起使用，这个时候可以不输出消耗时间。
    :param isDisplayTimeConsumed {type: bool} {default: False}: 是否显示消耗的时间，默认值不显示。
    :return None {count: 0}: 
    """
    if isDisplayTimeConsumed==True:
        print('Time consumed:', str(datetime.datetime.now() - startTime).split('.')[0])
    print("Completed at " + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ".\n")
    print("-------------------------Completed-------------------------")

def DisplaySeparator():
    print("---------------------------------------------------------\n")

可视化设置

In [ ]:
# 设置字体
# STFANGSO.TTF字体需要拷贝。在没有STFANGSO.TTF的情况下，可以使用windows默认字体simsun.ttc。
myFont = fm.FontProperties(fname='c:\windows\Fonts\STFANGSO.TTF')
# myFont = fm.FontProperties(fname='c:\windows\Fonts\simsun.ttc')

# 设置图像大小
figSizeNormal = (5, 3.09)
figSizeHuge = (8, 8)


def setRainbowColors(valueList):
    """_summary_ 设置饼图的彩虹色

    Args:
        valueList (list): 输入值的列表。

    Returns:
        list: 彩虹色的列表。
    """
    return cm.rainbow_r(np.arange(len(valueList))/len(valueList))

def make_autopct(values_list):
    def my_autopct(pct):
        total = sum(values_list)
        val = int(round(pct*total)/100.0)
        return '{p:1.1f}% ({v:d}人)'.format(p=pct, v=val)
    return my_autopct

# 首先计算百分比
def calculatePercentage(inputList):
    percentageList = []
    total = sum(inputList)
    for temp in inputList:
        percentageList.append(round((temp/total * 100.0),1))
    return percentageList

## 3. 数据读取及数据预处理

## 3.1 文件批量改名

|编号|数据名称|重命名英文名称|说明|
|---|---|---|---|
|1|附件1：长春市COVID-19疫情期间病毒感染人数数据-新增本土感染者|1nlip|1New Local Infected Persons|
|2|附件1：长春市COVID-19疫情期间病毒感染人数数据-新增无症状感染者|1nai|1New Asymptomatic Infections|
|3|附表2：长春市9个区隔离人口数量与生活物资投放点数量|2pndpl|2Population and Number of Distribution Points of LivingMaterials|
|4|交通路口节点数据|3tind|Traffic intersection node data|
|5|交通路口路线数据|3tird|Traffic intersection route data|
|6|各区主要小区数据|3dmced|Data of main communities in each district|
|7|附件3/1.XX月XX日长春市重点民生商品供应情况表(0316模板）|31sklc|Supply of Key Livelihood Commodities|
|8|附件3/2.长春市生活物资保障体系基本情况表|32bilmss|Basic Information of Living Material Support System|
|9|附件3/3.XX月XX日长春市主要粮油副食品零售价格监测表|33rpmconsf|Monitoring Table for Retail Price of Major Cereals, Oils and Non staple Foods|
|10|附件3/4.XX月XX日长春市重点蔬菜零售价格监测表|34rpkv|Monitoring Table for Retail Price of Key Vegetables|
|11|附件3/5.点调重点企业主要生活必需品价格情况表|35pmnke|Price of main necessities of key enterprises|
|12|附件3/|||
|13||||
|14||||
|15||||
|16||||
|||||
|||||
|||||

记录统计

|编号|日期|包含的数据文件|说明|
|---|---|---|---|
|1|0318|1,2,3,4,5||
|2|0319|1,2,3,4,5||
|3|0320|1,2||
|4|0401|1,2,111||
|5|0402|||
|6|0403|||
|7|0405|||
|8||||
|9||||
|10||||
|||||
|||||
|||||


1. “4月29日”是一个关键节点，同名的文件夹有2个数据。一份是正常的日报；另一份是复工情况报告，这一份重名为“4月29日fg”。
2. 一共有352个xls/xlsx文件。
3. 重点民生商品供应情况表共计57个。
4. 重点民生商品供应情况表 中定义列数有28列。定义行数为65行（也就是65天排除了4月29日fg这个文件夹）。
   1. 列名
      |编号|名称|英文|说明|
      |---|---|---|---|
      |1|粮食总储备量|gtr|Total grain reserves|
      |2|粮食政府储备|ggr|Grain government reserves|
      |3|粮食重点企业库存|gike|Inventory of key grain enterprises可以为空|
      |4|粮食日出库量|gdo|Daily grain output|
      |5|粮食日进货量|gdp|Daily grain purchase|
      |6|粮食保障天数|gsd|Grain security days|
      |7|粮食储备标准|grs|Grain reserve standard|
      |8|食用油总储备量|otr|Oil|
      |9|食用油政府储备|ogr||
      |10|食用油重点企业库存|oike|可以为空|
      |11|食用油日出库量|odo||
      |12|食用油日进货量|odp||
      |13|食用油保障天数|osd||
      |14|食用油储备标准|ors||
      |15|猪肉总储备量|ptr|pork|
      |16|猪肉政府储备|pgr||
      |17|猪肉重点企业库存|pike|可以为空|
      |18|猪肉日出库量|pdo||
      |19|猪肉日进货量|pdp||
      |20|猪肉保障天数|psd||
      |21|猪肉储备标准|prs||
      |22|蔬菜总储备量|vtr|vegatable|
      |23|蔬菜政府储备|vgr||
      |24|蔬菜重点企业库存|vike|可以为空|
      |25|蔬菜日出库量|vdo||
      |26|蔬菜日进货量|vdp||
      |27|蔬菜保障天数|vsd||
      |28|蔬菜储备标准|vrs||
   2. 行名：从'3月18日'开始，到'5月23日'结束。共65天，注意数据5月21日和5月22日缺失。。


生成列名列表。

In [35]:
sklcColoumnNameList = ['gtr', 'ggr', 'gike', 'gdo', 'gdp', 'gsd', 'grs', 
                       'otr', 'ogr', 'oike', 'odo', 'odp', 'osd', 'ors', 
                       'ptr', 'pgr', 'pike', 'pdo', 'pdp', 'psd', 'prs', 
                       'vtr', 'vgr', 'vike', 'vdo', 'vdp', 'vsd', 'vrs']

生成日期组成的列表。参考<https://www.cnblogs.com/cupleo/p/16290049.html>

In [34]:
def createDateList(startTime=None, endTime=None):
    """_summary_
    行名：从'3月18日'开始，到'5月23日'结束。共65天。注意数据5月21日和5月22日缺失。
    Args:
        startTime (str): 开始日期。只能是"2022-03-02"这种格式。
        endTime (str): 开始日期。只能是"2022-03-02"这种格式。

    Returns:
        list: 由日期字符串组成的列表。
    """
    
    if startTime is None:
        startTime = '2022-10-01'
    if endTime is None:
        endTime = datetime.datetime.now().strftime('%Y-%m-%d')
    
    startTime=datetime.datetime.strptime(startTime,'%Y-%m-%d')
    endTime=datetime.datetime.strptime(endTime,'%Y-%m-%d')
    
    date_list = []
    date_list.append(startTime.strftime('%Y-%m-%d'))
    while startTime < endTime:
        # 日期叠加一天
        startTime += datetime.timedelta(days=+1)
        # 日期转字符串存入列表
        date_list.append(startTime.strftime('%Y-%m-%d'))
    
    return date_list

sklcRowNameList = createDateList("2022-03-18", "2022-05-23")
print(sklcRowNameList)
print(len(sklcRowNameList))

['2022-03-18', '2022-03-19', '2022-03-20', '2022-03-21', '2022-03-22', '2022-03-23', '2022-03-24', '2022-03-25', '2022-03-26', '2022-03-27', '2022-03-28', '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01', '2022-04-02', '2022-04-03', '2022-04-04', '2022-04-05', '2022-04-06', '2022-04-07', '2022-04-08', '2022-04-09', '2022-04-10', '2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14', '2022-04-15', '2022-04-16', '2022-04-17', '2022-04-18', '2022-04-19', '2022-04-20', '2022-04-21', '2022-04-22', '2022-04-23', '2022-04-24', '2022-04-25', '2022-04-26', '2022-04-27', '2022-04-28', '2022-04-29', '2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03', '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07', '2022-05-08', '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-12', '2022-05-13', '2022-05-14', '2022-05-15', '2022-05-16', '2022-05-17', '2022-05-18', '2022-05-19', '2022-05-20', '2022-05-21', '2022-05-22', '2022-05-23']
67


1. 遍历文件参考:
    1. <https://blog.csdn.net/weixin_43115411/article/details/126049598>
    2. 清晰的说明。<http://coolpython.net/python_senior/pytip/iter_folder.html>
2. 复制文件参考<https://blog.csdn.net/m0_52650621/article/details/113405008>。使用shutil来对文件进行复制及重命名。
3. 对excel进行操作。<https://blog.csdn.net/qdPython/article/details/125603938>
4. python中os.path.join()函数：连接两个或更多的路径名函数。<https://blog.csdn.net/qq_40212975/article/details/108863273>
5. 获取文件后缀名，用第一个方法即可。非常好用。<https://blog.csdn.net/weixin_40895135/article/details/121033002>
6. 判断一个字符串中是否包含另一个字符串<https://jingyan.baidu.com/article/0964eca20d1bddc385f536db.html>
7. python的绝对时间起点和excel的绝对时间起点不一样。说明<https://zhuanlan.zhihu.com/p/143520836>。excel读到的日期的浮点数，代表的是自1900年1月1日以来的天数。python的时间戳是指格林威治时间自1970年1月1日至当前时间的总秒数。代码实现<https://blog.csdn.net/qq_42479394/article/details/116332683>。

In [98]:
# x = "政府储备已到期"
# y = "天地源米业740"
# z = [i for i in y if str.isdigit(i)]
# w = [i for i in x if str.isdigit(i)]
# print(w)
# s = "".join(z)
# print(int(s))

def GetExcelCellStringtoNumber(cell):
    """_summary_

    Args:
        cell (object): 可以是字符串、整型、浮点型。然后全部统一转换为浮点型。
                        将excel的cell中的数值提取出来。然后转化为浮点型输出。
                        对于"政府储备已到期"也可以理解为储备为0了。

    Returns:
        float: 输出一个浮点型数值，如果表格中这里是空的，那么返回0。
    """
    typecell = type(cell)
    # print(typecell)
    # print(type(typecell))
    if typecell == float or typecell == int:
        return cell

    isNumber = [i for i in cell if str.isdigit(i)]
    if len(isNumber) == 0:
        return 0
    
    strNumber = "".join(isNumber)
    return float(strNumber)

# rr = GetExcelCellStringtoNumber(y)

# print(type(rr))
# print(rr)


<class 'float'>
740.0


In [101]:
from xlrd import xldate_as_datetime

Appendix3Path = r"./data/附件4：长春市疫情期间每日生活物资相关数据/"

# 重点民生商品供应情况表 中定义列数有28列。定义行数为65行（也就是65天排除了4月29日fg这个文件夹），
# 注意数据5月21日和5月22日缺失。。
SklcPD = pd.DataFrame(index=sklcRowNameList, columns=sklcColoumnNameList)

totalExcelFilesNumber = 0
totalSklcFilesNumber = 0

for dirpath, dirnames, filenames in os.walk(Appendix3Path):
    for filename in filenames:
        temp = os.path.join(dirpath, filename)
        # print(temp)
        # print(type(temp))
        # print(temp[-4:])
        
        Suffix = os.path.splitext(temp)[-1]
        # print(Suffix)
        if Suffix == ".xls" or Suffix == ".xlsx":
            # print(temp)
            # 重点民生商品供应情况表(0316模板）  这个字符串太紧了，将3月20日等几天的剔除了。
            # 1.3月31日长春市重点民生商品供应情况表(0316模板）
            if '重点民生商品供应情况表' in temp:
                # print(temp)
                workbook = xlrd.open_workbook(temp)
                table = workbook.sheets()[0]

                rown = 6
                coln = 3

                cell = table.cell_value(rown - 1, coln - 1)
                # 将excel的时间转化为python的时间。xldate_as_datetime()返回的类型已经是字符串类型了。
                # print(xldate_as_datetime(cell, 0).strftime("%Y-%m-%d"))
                time = table.cell_value(3 - 1, 1 - 1)
                tableTime = xldate_as_datetime(time, 0).strftime("%Y-%m-%d")
                # print(type(tableTime))
                # print(tableTime)
                # SklcPD.loc[tableTime,'gtr'] = cell
                
                # 'gtr', 'ggr', 'gike', 'gdo', 'gdp', 'gsd', 'grs', 
                # 先将结果全部填进DataFrame里面，然后在统一修改类型和数值，并且去除杂质。
                SklcPD.loc[tableTime,'gtr'] = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 3 - 1))
                SklcPD.loc[tableTime,'ggr'] = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 4 - 1))
                gike1 = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 5 - 1))
                gike2 = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 6 - 1))
                gike3 = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 7 - 1))
                SklcPD.loc[tableTime,'gike'] = gike1 + gike2 + gike3
                SklcPD.loc[tableTime,'gdo'] = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 9 - 1))
                SklcPD.loc[tableTime,'gdp'] = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 10 - 1))
                SklcPD.loc[tableTime,'gsd'] = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 11 - 1))
                SklcPD.loc[tableTime,'grs'] = GetExcelCellStringtoNumber(table.cell_value(6 - 1, 12 - 1))
                
                # 'otr', 'ogr', 'oike', 'odo', 'odp', 'osd', 'ors', 
                SklcPD.loc[tableTime,'otr'] = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 3 - 1))
                SklcPD.loc[tableTime,'ogr'] = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 4 - 1))
                oike1 = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 5 - 1))
                oike2 = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 6 - 1))
                oike3 = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 7 - 1))
                SklcPD.loc[tableTime,'oike'] = oike1 + oike2 + oike3
                SklcPD.loc[tableTime,'odo'] = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 9 - 1))
                SklcPD.loc[tableTime,'odp'] = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 10 - 1))
                SklcPD.loc[tableTime,'osd'] = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 11 - 1))
                SklcPD.loc[tableTime,'ors'] = GetExcelCellStringtoNumber(table.cell_value(7 - 1, 12 - 1))
                
                # 'ptr', 'pgr', 'pike', 'pdo', 'pdp', 'psd', 'prs', 
                SklcPD.loc[tableTime,'ptr'] = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 3 - 1))
                SklcPD.loc[tableTime,'pgr'] = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 4 - 1))
                pike1 = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 5 - 1))
                pike2 = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 6 - 1))
                pike3 = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 7 - 1))
                SklcPD.loc[tableTime,'pike'] = pike1 + pike2 + pike3
                SklcPD.loc[tableTime,'pdo'] = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 9 - 1))
                SklcPD.loc[tableTime,'pdp'] = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 10 - 1))
                SklcPD.loc[tableTime,'psd'] = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 11 - 1))
                SklcPD.loc[tableTime,'prs'] = GetExcelCellStringtoNumber(table.cell_value(8 - 1, 12 - 1))

                # 'vtr', 'vgr', 'vike', 'vdo', 'vdp', 'vsd', 'vrs'
                SklcPD.loc[tableTime,'vtr'] = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 3 - 1))
                SklcPD.loc[tableTime,'vgr'] = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 4 - 1))
                vike1 = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 5 - 1))
                vike2 = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 6 - 1))
                vike3 = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 7 - 1))
                SklcPD.loc[tableTime,'vike'] = vike1 + vike2 + vike3
                SklcPD.loc[tableTime,'vdo'] = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 9 - 1))
                SklcPD.loc[tableTime,'vdp'] = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 10 - 1))
                SklcPD.loc[tableTime,'vsd'] = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 11 - 1))
                SklcPD.loc[tableTime,'vrs'] = GetExcelCellStringtoNumber(table.cell_value(9 - 1, 12 - 1))
                
                # print(cell)
                
                totalSklcFilesNumber += 1
            totalExcelFilesNumber += 1

print("totalExcelFilesNumber is {}. totalSklcFilesNumber is {}".format(totalExcelFilesNumber, 
                                                                       totalSklcFilesNumber))


totalExcelFilesNumber is 352. totalSklcFilesNumber is 57


In [102]:
SklcPD

,gtr,ggr,gike,gdo,gdp,gsd,grs,otr,ogr,oike,...,pdp,psd,prs,vtr,vgr,vike,vdo,vdp,vsd,vrs
2022-03-18,30179.0,15000.0,12800.0,152.0,82.0,21.0,15.0,4670.0,1800.0,2500.0,...,130.0,13.0,3.0,16679.0,10000.0,6610.0,1714.0,1400.0,7.0,7.0
2022-03-19,30123.0,15000.0,12700.0,128.0,47.0,21.0,15.0,4824.0,1800.0,2600.0,...,185.0,13.0,3.0,16337.0,10000.0,6320.0,1668.0,1237.0,7.0,7.0
2022-03-20,29982.0,15000.0,12700.0,238.0,73.0,21.0,15.0,4750.0,1800.0,2500.0,...,148.0,13.0,3.0,15889.0,10000.0,6150.0,1498.0,1126.0,7.0,7.0
2022-03-21,29935.0,15000.0,12700.0,130.0,69.0,20.0,15.0,4747.0,1800.0,2500.0,...,119.0,13.0,3.0,15521.0,10000.0,5770.0,1544.0,973.0,7.0,7.0
2022-03-22,29942.0,15000.0,12700.0,65.0,69.0,20.0,15.0,4744.0,1800.0,2500.0,...,119.0,13.0,3.0,15158.0,10000.0,6590.0,1443.0,956.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


将所有excel中的数据按日期读取出来，并统一建议按种类分布的表。

## 4. 建模

## 5. 可视化